# Start

In [2]:
import numpy as np
import pandas as pd

In [3]:
def generate_population():
    pass

In [18]:
def get_had_dataframe(
    a: np.array,
    b: np.array,
    v_a: str = 'A',
    v_b: str = 'B',
    feature: str = 'value'
) -> pd.DataFrame:
    a = pd.Series(a)
    b = pd.Series(b)
    size_nonnull_a = a.count()
    size_nonnull_b = b.count()

    if all((not size_nonnull_a, not size_nonnull_b)):
        raise ValueError('Both arrays are empty.')
    if not size_nonnull_a:
        raise ValueError('Array A is empty.')
    if not size_nonnull_b:
        raise ValueError('Array B is empty.')

    df_a = pd.DataFrame(a, columns=[feature])
    df_b = pd.DataFrame(b, columns=[feature])

    df_a = df_a.groupby(feature, as_index=False)\
        .size()\
        .rename(columns={'size': f'count_{v_a}'})
    df_a[f'bin_fraction_{v_a}'] = df_a[f'count_{v_a}'] / size_nonnull_a

    df_b = df_b.groupby(feature, as_index=False)\
        .size()\
        .rename(columns={'size': f'count_{v_b}'})
    df_b[f'bin_fraction_{v_b}'] = df_b[f'count_{v_b}'] / size_nonnull_b

    df_had = df_a.merge(df_b, how='outer', on=feature).fillna(0)
    df_had['diff'] = df_had[f'bin_fraction_{v_b}'] - df_had[f'bin_fraction_{v_a}']
    df_had['half_abs_diff'] = df_had['diff'].abs() / 2
    return df_had


In [25]:
# a = ['a', 'a', 'a', 'b', 'b', 'c', 'c', 'c', 'c', 'c', 'c', 'd']
# b = ['a', 'a', 'a', 'a', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'd', 'd']
# a = [None]
# b = [None]
a = [1,1,1]
b = [None, None, 1, 1]

In [22]:
df_had = get_had_dataframe(a, b)
df_had

,value,count_A,bin_fraction_A,count_B,bin_fraction_B,diff,half_abs_diff
0,1.0,3.0,1.0,2,0.5,-0.5,0.25
1,2.0,0.0,0.0,2,0.5,0.5,0.25


In [24]:
epsilon = 10**-9
df_had['ratio'] = df_had[f'bin_fraction_{v_b}'] / (df_had[f'bin_fraction_{v_a}'] + epsilon)
df_had['psi'] = df_had['diff'] * np.log(df_had['ratio'])
df_had

,value,count_A,bin_fraction_A,count_B,bin_fraction_B,diff,half_abs_diff,ratio,psi
0,1.0,3.0,1.0,2,0.5,-0.5,0.25,0.5,0.346574
1,2.0,0.0,0.0,2,0.5,0.5,0.25,500000000.0,10.015059


# End